In [62]:
import torch
import math
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset, Subset


from transformer_batch_corrections import Correction_data, make_resampled_dataset
from asses_batch_effect import batchless_entropy_estimate, abs_effect_estimate, fisher_kldiv, compute_F_stat

In [32]:
# CrossTab_name = 'synthetic_data_batchPercent_60_groupPercent_10'
CrossTab_name = 'peptide_crosstab_1'
PATH_TO_DATA   = 'data/' + CrossTab_name + '.txt'
CrossTab = pd.read_csv(PATH_TO_DATA,   delimiter = '\t')
minibatch_size = 60
megabatch_size = 50
batch_size = 10
n_batches = 6
reg_factor = 0.5
depth = 1
n_divs = [2]
run_name = CrossTab_name + "_ndivs_{0}_dep_{1}_reg_{2}_mb_{3}_rs_{4}".format(n_divs, depth, reg_factor, minibatch_size, megabatch_size)

In [38]:
test = Correction_data(CrossTab = CrossTab, depth = depth, reg_factor = reg_factor, 
                       n_batches = n_batches, batch_size = batch_size, test_size = 10000, 
                       minibatch_size = minibatch_size, random_state = 69, n_divs = n_divs)

In [ ]:
test.train_model(epochs = 21, report_frequency = 15, abs_effect_cutoff = 0, resample_training = megabatch_size, run_name = run_name, finetune_training = False)

In [ ]:
test.train_model(epochs = 1001, report_frequency = 20, abs_effect_cutoff = 0, resample_training = megabatch_size, run_name = run_name, finetune_training = True)

In [40]:
len(test.CrossTab)

80446

In [ ]:
test.compute_correction(test.TRAIN_DATA[2])

In [67]:
len(test.TRAIN_DATA)

70446

In [96]:
N

62616.0

In [95]:
window = 2/3
n_overlap = 3
loader = torch.utils.data.DataLoader(test.TRAIN_DATA, shuffle = False, batch_size = megabatch_size)
step = len(test.TRAIN_DATA)*window//n_overlap
window_N = step*n_overlap
N = (len(test.TRAIN_DATA)//step)*step

tensor_crosstab = []
for _, _, y, mask in loader:
    y, z = test.compute_correction(y, mask)
    tensor_crosstab.append(y-z)

tensor_crosstab = torch.cat(tensor_crosstab)
F_stat = compute_F_stat(tensor_crosstab, test.n_batches, test.batch_size)
F_df = pd.DataFrame({'index' : range(0, len(F_stat)), 'F_stat' : F_stat.cpu().detach().numpy()})

A = F_df.sort_values(by = 'F_stat', ascending = False)
B = pd.concat([A,A])


In [ ]:
[B['index'].to_list()[i*step:(n_overlap + i)*step] for i in range(0, n_div)]

In [92]:
B.iloc[70446]

index     64191.000000
F_stat      296.271459
Name: 64191, dtype: float64

In [83]:
window = 2/3
n_overlap = 3
loader = torch.utils.data.DataLoader(test.TRAIN_DATA, shuffle = False, batch_size = megabatch_size)
step = len(test.TRAIN_DATA)*window//n_overlap
tensor_crosstab = []
for _, _, y, mask in loader:
    y, z = test.compute_correction(y, mask)
    tensor_crosstab.append(y-z)

tensor_crosstab = torch.cat(tensor_crosstab)
F_stat = compute_F_stat(tensor_crosstab, test.n_batches, test.batch_size)
F_df = pd.DataFrame({'index' : range(0, len(F_stat)), 'F_stat' : F_stat.cpu().detach().numpy()})

C = F_df.sort_values(by = 'F_stat', ascending = False)

section_loaders = [C['index'].to_list()[i*div_size:(1+i)*div_size] for i in range(0, n_div)]
for index, section in enumerate(section_loaders):
    section = self.CrossTab.iloc[section]
    resampled_data = make_resampled_dataset(CrossTab = section, n_batches = self.n_batches, 
                                            minibatch_size = self.minibatch_size)
    resampled_loader = torch.utils.data.DataLoader(resampled_data, shuffle = True, batch_size = megabatch_size)
    section_loaders[index] = resampled_loader
self.finetune_loaders = self.finetune_loaders + section_loaders
self.batchless_entropy_distributions = self.batchless_entropy_distributions + batchless_entropy_distribuions(self.n_batches, self.batch_size, n_div)


NameError: name 'n_div' is not defined

In [61]:
C[0:2000000]

,index,F_stat,batch_dist
2010733,2010733,9.407303,-15.152463
4885414,4885414,9.222904,-14.840519
2808282,2808282,9.208838,-14.816668
95971,95971,8.721045,-13.984584
529082,529082,8.672985,-13.902085
...,...,...,...
3489478,3489478,1.036300,-0.552684
135350,135350,1.036300,-0.552684
3397633,3397633,1.036300,-0.552684
4199563,4199563,1.036300,-0.552684


In [45]:
natural_distributions[4]

[-1.1230517666577784,
 -1.1230532144609078,
 -1.1230535427275044,
 -1.1230541810214878,
 -1.12305497335295,
 -1.1230559264345485,
 -1.1230572751724308,
 -1.1230578145750556,
 -1.1230592674666107,
 -1.1230598347952307,
 -1.1230602276786725,
 -1.1230615968380429,
 -1.1230617726634025,
 -1.1230624161113831,
 -1.1230632522190627,
 -1.123063531368861,
 -1.1230638595882514,
 -1.1230639789681058,
 -1.1230648008442632,
 -1.1230654420834298,
 -1.123066759151751,
 -1.1230696666171598,
 -1.1230701192559058,
 -1.1230706158434693,
 -1.1230706395683594,
 -1.1230707321809397,
 -1.1230710010155027,
 -1.1230710448850334,
 -1.1230720287892084,
 -1.123072857059701,
 -1.1230735102140783,
 -1.1230746266674463,
 -1.1230752630101621,
 -1.1230756123457488,
 -1.123076358378818,
 -1.1230772107067652,
 -1.1230779499617118,
 -1.1230790093973595,
 -1.1230794479239106,
 -1.1230806219739873,
 -1.123081741171461,
 -1.123081820142013,
 -1.1230827737335494,
 -1.1230837996504421,
 -1.1230840033319178,
 -1.12308429096442

In [152]:
n_div = 5
div_size = len(test.loader.dataset)//n_div
tensor_crosstab = []
for _, _, y, mask in test.loader:
    y, z = test.compute_correction(y, mask)
    tensor_crosstab.append(y-z)

tensor_crosstab = torch.cat(tensor_crosstab)
individual_distance = fisher_kldiv(tensor_crosstab, test.n_batches, test.batch_size, 0)
individual_distance = pd.DataFrame({'index' : range(0, len(individual_distance)),
                                    'distance' : individual_distance.cpu().detach().numpy()})

C = individual_distance.sort_values(by = 'distance', ascending = False)

section_loaders = [C['index'].to_list()[i*div_size:(1+i)*div_size] for i in range(0, n_div)]
for index, section in enumerate(section_loaders):
    section = test.CrossTab.iloc[section]
    resampled_data = make_resampled_dataset(CrossTab = section, n_batches = test.n_batches, 
                                            minibatch_size = test.minibatch_size, n_stack = 10, 
                                            test_size = 0, random_state = test.random_state)
    resampled_loader = torch.utils.data.DataLoader(resampled_data, shuffle = True, batch_size = megabatch_size)
    section_loaders[index] = resampled_loader


In [164]:
C

,index,distance
2217,2217,94.842960
7957,7957,93.501318
13167,13167,92.843862
14982,14982,92.173830
2497,2497,91.536665
...,...,...
13129,13129,0.273005
9613,9613,0.273005
3223,3223,0.273005
6044,6044,0.273005


In [168]:
fisher_kldiv(next(iter(section_loaders[4]))[0], test.n_batches, test.batch_size, 0)

tensor([[0.3173, 0.3033, 0.2734,  ..., 0.2776, 0.2882, 0.3262],
        [0.3193, 0.5201, 0.2735,  ..., 0.2825, 0.3190, 0.3861],
        [0.3359, 0.3777, 0.4933,  ..., 0.2753, 0.3312, 0.3583],
        ...,
        [0.4717, 0.2962, 0.3262,  ..., 0.2749, 0.4243, 0.4645],
        [0.3218, 0.4836, 0.2818,  ..., 0.3758, 0.2823, 0.4619],
        [0.3176, 0.2951, 0.3854,  ..., 0.2737, 0.2949, 0.3212]],
       dtype=torch.float64)

In [101]:
fisher_kldiv(sections[4][2515][2], test.n_batches, test.batch_size, test.batchless_entropy)

tensor(-0.5547, dtype=torch.float64)

In [79]:
sections[0]

[2217,
 7957,
 13167,
 14982,
 2497,
 13721,
 231,
 554,
 2642,
 13593,
 6414,
 6608,
 11223,
 11927,
 731,
 12195,
 12389,
 11184,
 6927,
 12272,
 7460,
 2783,
 13285,
 8271,
 9129,
 7225,
 13494,
 10075,
 1498,
 10820,
 7587,
 3124,
 3626,
 12648,
 1639,
 12190,
 11807,
 8454,
 1914,
 570,
 10241,
 2358,
 14700,
 3965,
 6756,
 5540,
 6822,
 2268,
 1973,
 1663,
 310,
 4299,
 6274,
 14787,
 12957,
 9704,
 3013,
 10507,
 13196,
 9526,
 5369,
 7979,
 3455,
 7447,
 13609,
 13375,
 5015,
 116,
 4430,
 3677,
 13362,
 12671,
 4128,
 9593,
 10096,
 14688,
 6579,
 11244,
 4910,
 1491,
 14172,
 14441,
 8884,
 14338,
 7809,
 5156,
 14334,
 4013,
 5806,
 14091,
 2273,
 5477,
 1191,
 2798,
 14199,
 7787,
 14279,
 14370,
 5956,
 5876,
 3073,
 10575,
 12914,
 14265,
 12379,
 9801,
 337,
 13181,
 2336,
 1872,
 8137,
 6474,
 12049,
 3722,
 12945,
 14800,
 2607,
 9647,
 2480,
 9089,
 2200,
 11394,
 7765,
 12280,
 14103,
 8425,
 5255,
 1743,
 9239,
 1507,
 31,
 8621,
 1046,
 8484,
 11647,
 11175,
 11371

In [ ]:
divisions = 5
crosstab = []
for _, _, y, mask in test.loader:
    y, z = test.compute_correction(y, mask)
    crosstab.append(y-z)

crosstab = torch.cat(crosstab)
individual_distance = fisher_kldiv(crosstab, test.n_batches, test.batch_size, test.batchless_entropy)
individual_distance = pd.DataFrame({'index' : range(0, len(individual_distance)),
                                    'distance' : individual_distance.cpu().detach().numpy()})

C = individual_distance.sort_values(by = 'distance', ascending = True)

A = C['index'].to_list()[:len(C)//2]
B = C['index'].to_list()[len(C)//2:]

A = random.sample(A, k = len(A)) 
B = random.sample(B, k = len(A))
n_minibatch = len(loader)
A_last = 0
B_last = 0

new_order = []
for minibatch_i in range(0, n_minibatch):
    if (minibatch_i % 2 == 0):
        new_minibatch = A[A_last:A_last + math.floor(A_prop * self.minibatch_size)] + B[B_last:B_last + math.ceil(B_prop * self.minibatch_size)]
        A_last += math.floor(A_prop * self.minibatch_size)
        B_last += math.ceil(B_prop * self.minibatch_size)
        new_order += new_minibatch
    else:
        new_minibatch = A[A_last:A_last + math.ceil((1 - A_prop) * self.minibatch_size)] + B[B_last:B_last + math.floor((1 - B_prop) * self.minibatch_size)]
        A_last += math.ceil(B_prop * self.minibatch_size)
        B_last += math.floor(A_prop * self.minibatch_size)
        new_order += new_minibatch

# print(new_order[0:10])
dataset_shuffled = Subset(loader.dataset, indices = new_order)
new_loader = torch.utils.data.DataLoader(dataset_shuffled, shuffle = True, batch_size = self.minibatch_size)
return(new_loader)